**parepare training data for baseline features**

In [2]:
import sys,json,re,string
from typing import Callable, Dict, List, Set, Tuple
from elasticsearch import Elasticsearch
import gensim.downloader as api
import gensim
import datetime
import csv
sys.path.insert(1, 'extract_features')
sys.path.insert(1, 'util')
from helper_function import preprocess
from extract_No1to5_features_optimized import extract_features_1to5
from extract_No11to12_features_optimized import extract_features_11to12, get_collection_paramter
from extract_No13to15_features import TypeTaxonomy, extract_features_13to15
from extract_No16_feature import extract_features_16
from extract_No17to19_features import get_analyze,extract_features_17to19
from extract_No20to22_features import extract_features_20to22
from extract_No23to25_features_optimized import extract_features_23to25

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\junec\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\junec\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
# Reduce logging level.
import logging
logging.basicConfig(level=logging.ERROR)

logging.getLogger("elasticsearch").disabled = True

es= Elasticsearch(timeout=600)
es.info()

{'name': 'LAPTOP-ADBLIUPR',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': '5VELmBwJTk-urTuhZdTgew',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'zip',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [4]:
!which python
!which pip

/c/Users/junec/anaconda3/python
/c/Users/junec/anaconda3/Scripts/pip


In [5]:
#load files
filepath="data/DBpedia_map_type_entities.json"
with open(filepath, 'r', encoding='utf-8') as file:
    DBpedia_map_type_entities = json.load(file)
    
filepath="data/training_types.json"
with open(filepath,encoding='utf-8') as json_file:
    training_map_type_questions = json.load(json_file)

filepath="data/ElasticSearch_map_type_docID.json"
with open(filepath, 'r',encoding='utf-8') as f:
    docID_DBOtype_dict = json.load(f)
    
filepath="data/DBpedia_map_docID_docLength.json"
with open(filepath, 'r',encoding='utf-8') as f:
     map_docID_docLength = json.load(f)

filepath="../smart-dataset/datasets/DBpedia/smarttask_dbpedia_train.json"
with open(filepath, 'r') as f:
    smarttask_dbpedia_train = json.load(f)
    

typeobj=TypeTaxonomy("data/dbpedia_types.tsv")

In [6]:
def extract_features_baseline(
    question:str,
    dp_type: str,
    DBpedia_map_type_entities:Dict,
    docID_DBOtype_dict:Dict,
    map_docID_docLength:Dict,
    collection_len:int,doc_number:int,avgdl:float,
    es: Elasticsearch,
 
) -> List[float]:
    """Extracts features of a query and document pair.

        Args:
            query: string.
            dp_type: DBO type.
            DBpedia_map_type_entities:DBOtype and all the DBpedia entities having that type.
            docID_DBOtype_dict:dictionary for elasticseach doc_id and DBOtype.
            es: Elasticsearch object instance.

        Returns:
            List of extracted feature values in a fixed order.
    """
    features_1to5 = extract_features_1to5(DBpedia_map_type_entities,dp_type,question,es)
    feature_vect = list(features_1to5.values())

    features_11to12 = extract_features_11to12(dp_type, question,
                                              docID_DBOtype_dict,
                                              map_docID_docLength,
                                              collection_len,doc_number,avgdl,
                                              es)
    feature_vect.extend(list(features_11to12.values()))
    
    return feature_vect

In [7]:
#with first path filter
def prepare_training_data_baseline( smarttask_dbpedia_train:List[Dict],
                            DBpedia_map_type_entities:Dict[str,List],
                            docID_DBOtype_dict:Dict[str,str],
                            map_docID_docLength:Dict[str,int], 
                            es: Elasticsearch
                            )-> Tuple[List[List[float]], List[int]]:
    """X and y label for base line features. Save to csv file.
         Args:
            smarttask_dbpedia_train: smarttask training set.
            DBpedia_map_type_entities:DBOtype and all the DBpedia entities having that type.
            docID_DBOtype_dict:dictionary for elasticseach doc_id and DBOtype.
            es: Elasticsearch object instance.

    """


    collection_len,doc_number,avgdl=get_collection_paramter(es,field="abstract",index="dbpdiea_type_centric")
    
    
    count=0
    j=0
    print("-----------------",datetime.datetime.now())
    with open("data/for_training_baseline.csv", 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)


        for entry in smarttask_dbpedia_train:
 
            if entry['question']==None:
                continue

            question_processed=preprocess(entry['question'])

            if entry['category']=='resource':
                if count%30==0:
                    print("--------------------count:",count)
                count+=1
                #print("question_processed:",question_processed)
                for DBOtype in entry['type']: 
                    try:
                        features=extract_features_baseline(question_processed,
                                                            DBOtype,
                                                            DBpedia_map_type_entities,
                                                            docID_DBOtype_dict,
                                                            map_docID_docLength,                                                
                                                            collection_len,doc_number,avgdl,
                                                            es)
                    except BaseException as err:
                        print("------------error for type:",DBOtype,entry['question'])
                        print(f"Unexpected {err}, {type(err)}")  
                        raise
                    j+=1
                    if j%500==0:
                        print("-----------------",datetime.datetime.now())
                        print("--------------------j:",j)
                    writer.writerow(features)
                    writer.writerow([1])


                #deal with top 30 documents
                hits = es.search(
                    index="dbpdiea_type_centric", q=question_processed, _source=True, size=30
                )["hits"]["hits"]
                rank_list= [hit['_source']["type"] for hit in hits]
            

                for DBOtype in rank_list:
                    if DBOtype not in entry['type']:
                        try:
                            features=extract_features_baseline(question_processed,
                                                                DBOtype,
                                                                DBpedia_map_type_entities,
                                                                docID_DBOtype_dict,
                                                                map_docID_docLength,                                                
                                                                collection_len,doc_number,avgdl,
                                                                es)
                        except BaseException as err:
                            print("------------error for type:",DBOtype,entry['question'])
                            print(f"Unexpected {err}, {type(err)}")  
                            raise
                            
                        j+=1
                        if j%500==0:
                            print("-----------------",datetime.datetime.now())
                            print("--------------------j:",j)
                        writer.writerow(features)
                        writer.writerow([0])


        
    print(f'total number {j} questions have been processed, in which {count} question has resource category')

    return None


In [8]:
prepare_training_data_baseline(smarttask_dbpedia_train,
                                        DBpedia_map_type_entities,
                                        docID_DBOtype_dict,
                                        map_docID_docLength,
                                        es) 

C:\Users\junec\anaconda3\lib\site-packages\elasticsearch\connection\base.py:209: ElasticsearchWarning: [types removal] Specifying types in term vector requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


--------------------count: 0
------------error for type: dbo:Book Who is the pupil of the tutor Miguel Ángel Estrella?
Unexpected , <class 'KeyboardInterrupt'>



KeyboardInterrupt

